# Redes Neurais - Projeto 3
# Modelos Generativos - Autoencoders Variacionais
---------------------
### Luis Filipe Menezes
#### RA: 164924

## 1. Objetivos:
Este caderno consiste na terceira entrega da disciplina de Redes Neurais realizada no programa de Pós Graduação em Ciência da Computação durante meu mestrado.

O projeto tem como objetivo:

- Selecionar 2 datasets (rotulados)

  - Treinar modelos VAEs:
  
  - Ajustar o melhor modelo (topologia) segundo a função de custo (conjunto validação)


- Explorar o espaço latente:

  - Gerar gráficos com a projeção do espaço latente em 2D (PCA)

  - Usar os rótulos na projeção.

- Algumas questões:

  1. Há formação de clusters no espaço latente?
  
  2. Há separação dos rótulos no espaço latente?
  
  3. A projeção ilustra quanto da variância?

- Adicional (opcional): Enviesar a formação do espaço latente com os exemplos rotulados


Além disso, tentarei utilizar o PyTorch nesse projeto. A implementação será feita se baseando neste [material](https://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/08-deep-autoencoders.html).

Como o modelo que ele apresenta utiliza uma CNN no dataset Ciphar, como ainda não foi estudado CNNs irei utilizar uma MLP no dataset Fashion MNIST.


### Bibliotecas utilizadas: 

In [6]:
from torchvision.datasets import FashionMNIST
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

## Dataset utilizado (Fashion MNIST)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])